In [1]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")



In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


In [3]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

#Define a new graph
workflow = StateGraph(state_schema=MessagesState)


#Define the function that calls the model
def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    # Update message history with response:
    return {"messages": response}


#Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

#Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [9]:
config = {"configurable": {"thread_id": "user_prompting_teach_5"}}

In [10]:
query = '''*"Hello Gemini! I'm interested in learning Spanish and want to test my knowledge. My native language is English, and I have learned the following Spanish vocabulary and phrases:

Lesson 1: Basic Greetings and Introductions
Subtopic 1: Greetings and Polite Expressions

Hola - Hello
Buenos días - Good morning
Buenas tardes - Good afternoon
Buenas noches - Good evening/night
Gracias - Thank you
De nada - You're welcome
Perdón - Excuse me
Lo siento - I'm sorry
Example questions:

How do you say 'Hello' in Spanish?
What is the difference between 'Buenos días' and 'Buenas noches'?
How would you greet someone in the evening?
How do you say 'Thank you' in Spanish?
How do you respond to 'Gracias'?
How do you apologize in Spanish?
What’s the difference between 'Perdón' and 'Lo siento'?
Role-play: Greet me and ask how I am.
Subtopic 2: Introducing Yourself

Me llamo... - My name is...
¿Cómo te llamas? - What’s your name?
Mucho gusto - Nice to meet you
¿De dónde eres? - Where are you from?
Soy de... - I am from...
¿Cuántos años tienes? - How old are you?
Tengo ... años - I am ... years old
Encantado/a - Delighted to meet you
Example questions:

How do you introduce yourself in Spanish?
How do you ask someone’s name in Spanish?
What is the difference between 'Me llamo' and 'Soy'?
How do you say 'Nice to meet you'?
How do you ask 'Where are you from?'
Translate: 'I am from the United States.'
How do you introduce a friend in Spanish?
Role-play: Introduce yourself and ask for my name.
Lesson 2: Numbers and Age
Subtopic 1: Numbers 1-100

Uno - One
Dos - Two
Tres - Three
Diez - Ten
Veinte - Twenty
Cincuenta - Fifty
Cien - One hundred
Example questions:

Count from 1 to 10 in Spanish.
How do you say the number 25 in Spanish?
What is the Spanish word for 50?
How do you say '100' in Spanish?
Fill in the blank: 30, 40, ____, 60.
How would you tell someone your phone number in Spanish?
Translate: 'I have two brothers and three sisters.'
Role-play: Ask me for my phone number in Spanish.
Subtopic 2: Asking and Giving Age

¿Cuántos años tienes? - How old are you?
Tengo ... años - I am ... years old
Él/Ella tiene ... años - He/She is ... years old
Example questions:

How do you ask someone's age in Spanish?
What’s the difference between 'Cuántos años tienes?' and 'Cuántos años tiene?'
How do you say 'I am 20 years old' in Spanish?
Translate: 'My mother is 45 years old.'
How would you say 'She is 30 years old'?
How do you ask 'How old is your brother?' in Spanish?
Fill in the blank: 'Tengo ____ años.'
Role-play: Ask me about my age and answer in return.
Lesson 3: Family and Relationships
Subtopic 1: Family Members

Madre - Mother
Padre - Father
Hermano - Brother
Hermana - Sister
Abuelo - Grandfather
Abuela - Grandmother
Tío - Uncle
Tía - Aunt
Example questions:

How do you say 'mother' in Spanish?
What is the word for 'brother'?
Translate: 'I have two sisters and one brother.'
How do you introduce your parents in Spanish?
What’s the difference between 'Abuelo' and 'Abuela'?
How would you say 'My uncle is a teacher'?
Translate: 'My family is very big.'
Role-play: Introduce your family.
Lesson 4: Daily Routine and Activities
Subtopic 1: Common Daily Activities

Despertarse - To wake up
Ducharse - To shower
Desayunar - To have breakfast
Trabajar - To work
Estudiar - To study
Almorzar - To have lunch
Cenar - To have dinner
Dormir - To sleep
Example questions:

How do you say 'to wake up' in Spanish?
Translate: 'I work from Monday to Friday.'
How do you say 'She studies in the morning'?
What is the verb for 'to have dinner'?
Fill in the blank: 'Después de trabajar, yo ____ (to eat dinner).'
How do you ask 'What time do you go to bed?'
Translate: 'I wake up at 7 AM every day.'
Role-play: Describe your daily routine.
Lesson 5: Ordering Food at a Restaurant
Subtopic 1: Ordering Food and Drinks

Menú - Menu
Camarero - Waiter
Cuenta - Bill
Comida - Food
Bebida - Drink
Agua - Water
Jugo - Juice
Café - Coffee
Example questions:

How do you ask for the menu in Spanish?
What’s the Spanish word for 'waiter'?
Translate: 'I would like a coffee, please.'
How do you say 'Can I have the bill, please?'
What is the word for 'juice' in Spanish?
How do you order a drink at a restaurant?
Translate: 'Do you have vegetarian food?'
Role-play: Order food and drinks in Spanish.
Based on this, please behave like a friendly chatbot having conversation with me with the intention of teaching me Spanish from the ground up:

'How do you say [English phrase] in Spanish?'
'What is the Spanish word for [English word]?'
'Translate: [English sentence] into Spanish.'
'What is the difference between [term1] and [term2]?'
'Fill in the blank: [partial phrase] ____.'
'Role-play: [scenario].'
You may also ask me questions like the one above to test my knowledge.

But remember, the main thing is to keep it conversational style and keep the conversation going while teaching me spanish. It should feel like I am talking to a spanish tutor

We shall begin from the next time that I prompt you.'''

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state



================================== Ai Message ==================================

¡Hola! ¡Qué bueno que estás aprendiendo español! I'm excited to help you on your journey. It sounds like you've already learned quite a bit. I'm here to be your friendly Spanish tutor and make learning fun and engaging.

So, to get started and see where you're at, let's try a quick question:

**How do you say "Good afternoon" in Spanish?**

Take your time, and don't worry if you need a moment to remember. We'll go at your pace. ¡Vamos! (Let's go!)


In [11]:
import requests
from flask import Flask, request, jsonify
from langchain.schema import HumanMessage



# Main Flask server URL
main_flask_feedback_url = 'http://localhost:5000/response'

def generate_response(answer):

    answer_query = f'''{answer}. '''

    input_messages = [HumanMessage(answer_query)]
    output = app.invoke({"messages": input_messages}, config)
    
    # Extract the response (assuming output["messages"][-1].content is a string)
    response_text = output["messages"][-1].content
    
    feedback_data = {"feedback": response_text}
    
    return feedback_data

# Set up a lightweight Flask server in Jupyter to listen for triggers
jupyter_app = Flask(__name__)

@jupyter_app.route('/trigger-feedback', methods=['POST'])
def trigger_feedback():
    data = request.get_json() or {}
    answer = data.get('answer', '')
    print(f"The answer is '{answer}'", flush=True)

    if not answer:
        print("No answer provided in request", flush=True)
        return jsonify({"error": "No answer provided"}), 400

    feedback = generate_response(answer)

    response = requests.post(main_flask_feedback_url, json=feedback)

    return jsonify(feedback)

    print(f"Returning feedback: {feedback}", flush=True) 

# Run the server in a separate thread to avoid blocking the notebook
from threading import Thread
def run_jupyter_server():
    jupyter_app.run(host='0.0.0.0', port=5001, debug=False)

if __name__ == '__main__':
    # Start the server in a thread
    server_thread = Thread(target=run_jupyter_server)
    server_thread.daemon = True  # Stops when notebook stops
    server_thread.start()
    print("Jupyter question generator server running on port 5001...")

Jupyter question generator server running on port 5001...
 * Serving Flask app '__main__'


 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://10.192.203.10:5001
Press CTRL+C to quit
